<a href="https://colab.research.google.com/github/DiogoLepri/ASD2_Project/blob/main/ASD-DiagNet2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/ASD2_Project/

/content/drive/MyDrive/ASD2_Project


In [3]:
!pip install pandas numpy matplotlib scikit-learn torch pyprind scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

##Plotly installing


In [4]:
%cd plotly-roc
!python setup.py install

/content/drive/MyDrive/ASD2_Project/plotly-roc
running install
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.11/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
  

In [5]:
#options: cc200, dosenbach160, aal
p_ROI = "cc200"
p_fold = 10
p_center = "Stanford"
p_mode = "whole"
p_augmentation = True
p_Method = "ASD-DiagNet"

In [6]:
parameter_list = [p_ROI,p_fold,p_center,p_mode,p_augmentation,p_Method]
print("*****List of patameters****")
print("ROI atlas: ",p_ROI)
print("per Center or whole: ",p_mode)
if p_mode == 'percenter':
    print("Center's name: ",p_center)
print("Method's name: ",p_Method)
if p_Method == "ASD-DiagNet":
    print("Augmentation: ",p_augmentation)


*****List of patameters****
ROI atlas:  cc200
per Center or whole:  whole
Method's name:  ASD-DiagNet
Augmentation:  True


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce
from sklearn.impute import SimpleImputer
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import pyprind
import sys
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_predict
import torch.optim as optim
from scipy import stats
from sklearn import tree
import functools
import numpy.ma as ma # for masked arrays
import pyprind
import random
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report
from scipy.interpolate import interp1d
import plotly.graph_objects as go
from plotly_roc import metrics, graphs
import warnings
warnings.filterwarnings('ignore')

In [8]:
%cd /content/drive/MyDrive/ASD2_Project

/content/drive/MyDrive/ASD2_Project


## Importing the data

In [9]:
def get_key(filename):
    f_split = filename.split('_')
    if f_split[3] == 'rois':
        key = '_'.join(f_split[0:3])
    else:
        key = '_'.join(f_split[0:2])
    return key

In [10]:
data_main_path = f'/content/drive/MyDrive/ASD2_Project/Outputs/cpac/filt_global/rois_{p_ROI}'
flist = os.listdir(data_main_path)
print(len(flist))

for f in range(len(flist)):
    flist[f] = get_key(flist[f])


df_labels = pd.read_csv('/content/drive/MyDrive/ASD2_Project/Phenotypic_V1_0b_preprocessed1.csv')
df_labels.DX_GROUP = df_labels.DX_GROUP.map({1: 1, 2: 0})
print(len(df_labels))

labels = {}
for i, row in df_labels.iterrows():
    file_id = row['FILE_ID']
    y_label = row['DX_GROUP']
    if file_id == 'no_filename':
        continue
    assert(file_id not in labels)
    labels[file_id] = y_label

#new age
df_labels['AGE_AT_SCAN'] = df_labels['AGE_AT_SCAN'].astype(np.float32)
ages = df_labels['AGE_AT_SCAN']
ages_norm = ((ages - ages.mean()) / ages.std()).round(1)

#IQS
iq_columns = ['FIQ', 'PIQ', 'VIQ']


for col in iq_columns:
    df_labels.loc[df_labels[col] == -9999, col] = np.nan


imputer = IterativeImputer(random_state=0)
df_labels[iq_columns] = imputer.fit_transform(df_labels[iq_columns])


phen_dict = {}
for i, row in df_labels.iterrows():
    file_id = row['FILE_ID']
    if file_id == 'no_filename':
        continue
    phen_dict[file_id] = [ages_norm.iloc[i], row['FIQ'], row['VIQ'], row['PIQ']]
#phen_dict[file_id] = [ages_norm.iloc[i], row['FIQ'], row['VIQ'], row['PIQ']]

884
1112


### Helper functions for computing correlations

In [11]:
def get_label(filename):
    assert (filename in labels)
    return labels[filename]


def get_corr_data(filename):
    #print(filename)
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')

    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()

def get_corr_matrix(filename):
    for file in os.listdir(data_main_path):
        if file.startswith(filename):
            df = pd.read_csv(os.path.join(data_main_path, file), sep='\t')
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(df.T))
        return corr

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

def get_regs(samplesnames,regnum):
    datas = []
    for sn in samplesnames:
        datas.append(all_corr[sn][0])
    datas = np.array(datas)
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0)
    return regions


## Helper fnuctions for computing correlations (Step A)

In [12]:
if not os.path.exists('./correlations_file'+p_ROI+'.pkl'):
    pbar=pyprind.ProgBar(len(flist))
    all_corr = {}
    for f in flist:

        lab = get_label(f)
        all_corr[f] = (get_corr_data(f), lab)
        pbar.update()

    print('Corr-computations finished')

    pickle.dump(all_corr, open('./correlations_file'+p_ROI+'.pkl', 'wb'))
    print('Saving to file finished')

else:
    all_corr = pickle.load(open('./correlations_file'+p_ROI+'.pkl', 'rb'))

## Computing eigenvalues and eigenvector

In [13]:
if p_Method=="ASD-DiagNet":
    eig_data = {}
    pbar = pyprind.ProgBar(len(flist))
    for f in flist:
        d = get_corr_matrix(f)
        eig_vals, eig_vecs = np.linalg.eig(d)

        for ev in eig_vecs.T:
            np.testing.assert_array_almost_equal(1.0, np.linalg.norm(ev))

        sum_eigvals = np.sum(np.abs(eig_vals))
        # Make a list of (eigenvalue, eigenvector, norm_eigval) tuples
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i], np.abs(eig_vals[i])/sum_eigvals)
                     for i in range(len(eig_vals))]

        # Sort the (eigenvalue, eigenvector) tuples from high to low
        eig_pairs.sort(key=lambda x: x[0], reverse=True)

        eig_data[f] = {'eigvals':np.array([ep[0] for ep in eig_pairs]),
                       'norm-eigvals':np.array([ep[2] for ep in eig_pairs]),
                       'eigvecs':[ep[1] for ep in eig_pairs]}
        pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:02:30


## Calculating Eros similarity

In [14]:
def norm_weights(sub_flist):
    num_dim = len(eig_data[flist[0]]['eigvals'])
    norm_weights = np.zeros(shape=num_dim)
    for f in sub_flist:
        norm_weights += eig_data[f]['norm-eigvals']
    return norm_weights

def cal_similarity(d1, d2, weights, lim=None):
    res = 0.0
    if lim is None:
        weights_arr = weights.copy()
    else:
        weights_arr = weights[:lim].copy()
        weights_arr /= np.sum(weights_arr)
    for i,w in enumerate(weights_arr):
        res += w*np.inner(d1[i], d2[i])
    return res

## Defining dataset class (Step B)

In [15]:
class CC200Dataset(Dataset):
    def __init__(self, pkl_filename=None, data=None, samples_list=None,
                 phenotype_data=None,#new
                 augmentation=False, aug_factor=1, num_neighbs=5,
                 eig_data=None, similarity_fn=None, verbose=False, regs=None):
        self.regs = regs
        self.phenotype_data = phenotype_data  #new
        if pkl_filename is not None:
            if verbose:
                print('Loading ..!', end=' ')
            self.data = pickle.load(open(pkl_filename, 'rb'))
        elif data is not None:
            self.data = data.copy()
        else:
            sys.stderr.write('Either PKL file or data is needed!')
            return

        #if verbose:
        #    print ('Preprocess..!', end='  ')
        if samples_list is None:
            self.flist = [f for f in self.data]
        else:
            self.flist = [f for f in samples_list]
        self.labels = np.array([self.data[f][1] for f in self.flist])

        current_flist = np.array(self.flist.copy())
        current_lab0_flist = current_flist[self.labels == 0]
        current_lab1_flist = current_flist[self.labels == 1]
        #if verbose:
        #    print(' Num Positive : ', len(current_lab1_flist), end=' ')
        #    print(' Num Negative : ', len(current_lab0_flist), end=' ')


        if augmentation:
            self.num_data = aug_factor * len(self.flist)
            self.neighbors = {}
            pbar = pyprind.ProgBar(len(self.flist))
            weights = norm_weights(samples_list)#??
            for f in self.flist:
                label = self.data[f][1]
                candidates = (set(current_lab0_flist) if label == 0 else set(current_lab1_flist))
                candidates.remove(f)
                eig_f = eig_data[f]['eigvecs']
                sim_list = []
                for cand in candidates:
                    eig_cand = eig_data[cand]['eigvecs']
                    sim = similarity_fn(eig_f, eig_cand, weights)
                    sim_list.append((sim, cand))
                sim_list.sort(key=lambda x: x[0], reverse=True)
                self.neighbors[f] = [item[1] for item in sim_list[:num_neighbs]]#list(candidates)#[item[1] for item in sim_list[:num_neighbs]]

        else:
            self.num_data = len(self.flist)

    def __getitem__(self, index):
        if index < len(self.flist):
            fname = self.flist[index]
            data = self.data[fname][0].copy() #get_corr_data(fname, mode=cal_mode)
            data = data[self.regs].copy()
            label = (self.labels[index],)

            #new
            if self.phenotype_data is not None:
                # Retrieve phenotype data using the file identifier as key
                pheno_vals = self.phenotype_data[fname]
                return torch.FloatTensor(data), torch.FloatTensor(pheno_vals), torch.FloatTensor(label)
            else:
                return torch.FloatTensor(data), torch.FloatTensor(label)

        else:
            f1 = self.flist[index % len(self.flist)]
            d1, y1 = self.data[f1][0], self.data[f1][1]
            d1=d1[self.regs]
            if len(self.neighbors[f1]) > 0:#new
                f2 = np.random.choice(self.neighbors[f1])
            else:
                f2 = f1  # fallback to self if no neighbors exist
            d2, y2 = self.data[f2][0], self.data[f2][1]
            d2 = d2[self.regs]
            assert y1 == y2
            r = np.random.uniform(low=0, high=1)
            label = (y1,)
            data = r * d1 + (1 - r) * d2

            #new check---------
            if self.phenotype_data is not None:
                # Use phenotype from the first sample for the augmented data
                pheno_vals = self.phenotype_data[f1]
                return torch.FloatTensor(data), torch.FloatTensor(pheno_vals), torch.FloatTensor(label)
            else:
                return torch.FloatTensor(data), torch.FloatTensor(label)

    def __len__(self):
        return self.num_data

## Definig data loader function

In [16]:
def get_loader(pkl_filename=None, data=None, samples_list=None,
               batch_size=64,
               num_workers=1, mode='train',
               *, augmentation=False, aug_factor=1, num_neighbs=5,
               eig_data=None, similarity_fn=None, verbose=False, regions=None,
               phenotype_data=None):  #new pheno data
    """Build and return data loader."""
    if mode == 'train':
        shuffle = True
    else:
        shuffle = False
        augmentation = False

    dataset = CC200Dataset(pkl_filename=pkl_filename, data=data, samples_list=samples_list,
                           augmentation=augmentation, aug_factor=aug_factor,
                           eig_data=eig_data, similarity_fn=similarity_fn, verbose=verbose, regs=regions,
                           phenotype_data=phenotype_data)  #new pheno data

    data_loader = DataLoader(dataset,
                             batch_size=batch_size,
                             shuffle=shuffle,
                             num_workers=num_workers)

    return data_loader

## Defining Autoencoder class (Step C & D)

In [17]:
class MTAutoEncoder(nn.Module):
    def __init__(self, num_inputs=990, num_latent=200, tied=True,
                 num_classes=2, use_dropout=False,
                 use_age=True, use_fiq=True, use_viq=True, use_piq=True):
        super(MTAutoEncoder, self).__init__()
        self.tied = tied
        self.num_latent = num_latent

        # Feature selection flags
        self.use_age = use_age
        self.use_fiq = use_fiq
        self.use_viq = use_viq
        self.use_piq = use_piq

        # Calculate number of phenotypic features used
        self.num_pheno_features = 0
        if use_age: self.num_pheno_features += 1
        if use_fiq: self.num_pheno_features += 1
        if use_viq: self.num_pheno_features += 1
        if use_piq: self.num_pheno_features += 1

        self.fc_encoder = nn.Linear(num_inputs, num_latent)

        if not tied:
            self.fc_decoder = nn.Linear(num_latent, num_inputs)

        if use_dropout:
            self.classifier = nn.Sequential(
                nn.Dropout(p=0.5),
                nn.Linear(self.num_latent + self.num_pheno_features, 1)
            )
        else:
            self.classifier = nn.Sequential(
                nn.Linear(self.num_latent + self.num_pheno_features, 1)
            )

    def forward(self, x, pheno_data=None, eval_classifier=False):
        x = self.fc_encoder(x)
        x = torch.tanh(x)

        if eval_classifier:
            if pheno_data is None:
                raise ValueError("Classifier mode requires pheno values input")

            pheno_data = pheno_data.to(x.device)

            # Select only the features we want to use
            selected_features = []
            if self.use_age: selected_features.append(pheno_data[:, 0:1])
            if self.use_fiq: selected_features.append(pheno_data[:, 1:2])
            if self.use_viq: selected_features.append(pheno_data[:, 2:3])
            if self.use_piq: selected_features.append(pheno_data[:, 3:4])

            if selected_features:
                selected_pheno = torch.cat(selected_features, dim=1)
                combined = torch.cat((x, selected_pheno), dim=1)
            else:
                combined = x

            # Print the classifier input vector shape only once per cell
            if not hasattr(self, '_printed_vector_size'):
                feature_count = self.num_pheno_features if self.num_pheno_features > 0 else "No"
                print(f"Classifier using {feature_count} phenotypic features, shape:", combined.shape)
                self._printed_vector_size = True

            x_logit = self.classifier(combined)
        else:
            x_logit = None

        if self.tied:
            x = F.linear(x, self.fc_encoder.weight.t())
        else:
            x = self.fc_decoder(x)

        return x, x_logit

## Defining training and testing functions

In [18]:
def train(model, epoch, train_loader, p_bernoulli=None, mode='both', lam_factor=1.0):
    model.train()
    train_losses = []

    for i, (images, pheno_data, batch_y) in enumerate(train_loader):#new before batch_x now = images
        if len(images) != batch_size:
            continue

        if p_bernoulli is not None:
            if i == 0:
                p_tensor = torch.ones_like(images).to(device) * p_bernoulli
            rand_bernoulli = torch.bernoulli(p_tensor).to(device)

        images = images.to(device)
        pheno_data = pheno_data.to(device)#new
        batch_y = batch_y.to(device)
        optimizer.zero_grad()

        if mode in ['both', 'ae']:
            if p_bernoulli is not None:
                rec_noisy, _ = model(images * rand_bernoulli, pheno_data, False)#new
                loss_ae = criterion_ae(rec_noisy, images) / len(images)
            else:
                rec, _ = model(images, pheno_data, False)#new
                loss_ae = criterion_ae(rec, images) / len(images)

        if mode in ['both', 'clf']:
            rec_clean, logits = model(images, pheno_data, True)#new
            loss_clf = criterion_clf(logits, batch_y)

        if mode == 'both':
            loss_total = loss_ae + lam_factor * loss_clf
            train_losses.append([loss_ae.detach().cpu().numpy(), loss_clf.detach().cpu().numpy()])
        elif mode == 'ae':
            loss_total = loss_ae
            train_losses.append([loss_ae.detach().cpu().numpy(), 0.0])
        elif mode == 'clf':
            loss_total = loss_clf
            train_losses.append([0.0, loss_clf.detach().cpu().numpy()])

        loss_total.backward()
        optimizer.step()

    return train_losses

def test(model, criterion, test_loader, eval_classifier=False, num_batch=None, return_probs=False):
    test_loss, n_test, correct = 0.0, 0, 0
    all_predss = []
    if eval_classifier:
        y_true, y_pred = [], []
    with torch.no_grad():
        model.eval()
        for i, (images, pheno_data, batch_y) in enumerate(test_loader, 1):
            if num_batch is not None and i >= num_batch:
                continue
            images = images.to(device)
            pheno_data = pheno_data.to(device)
            batch_y = batch_y.to(device)

            rec, logits = model(images, pheno_data, eval_classifier)
            test_loss += criterion(rec, images).detach().cpu().numpy()
            n_test += len(images)
            if eval_classifier:
                proba = torch.sigmoid(logits).detach().cpu().numpy()
                preds = np.ones_like(proba, dtype=np.int32)
                preds[proba < 0.5] = 0
                all_predss.extend(preds)
                y_arr = batch_y.cpu().numpy().astype(np.int32)
                correct += np.sum(preds == y_arr)
                y_true.extend(y_arr.tolist())
                y_pred.extend(proba.tolist())
        if eval_classifier:
            mlp_acc, mlp_sens, mlp_spef = confusion(y_true, all_predss)

            if return_probs:
                return mlp_acc, mlp_sens, mlp_spef, y_true, y_pred
            else:
                return mlp_acc, mlp_sens, mlp_spef
    return None



##Ploting function


In [19]:
def plot_roc_curves(predictions_dict, title="ASD-DiagNet: ROC Curve Comparison"):
    """Create a single ROC curve plot with multiple curves"""

    import plotly.graph_objects as go
    from plotly_roc import metrics, graphs
    from sklearn.metrics import roc_curve, auc

    fig = go.Figure()

    colors = ["crimson", "royalblue", "forestgreen", "orange"]

    for i, (model_name, preds) in enumerate(predictions_dict.items()):
        # Calculate ROC curve metrics
        metrics_df = metrics.metrics_df(preds['y_true'], preds['y_prob'])

        # Calculate AUC
        fpr, tpr, _ = roc_curve(preds['y_true'], preds['y_prob'])
        roc_auc = auc(fpr, tpr)

        # Create the ROC curve using plotly-roc
        temp_fig = graphs.roc_curve(
            metrics_df,
            line_name=f"{model_name} (AUC: {roc_auc:.4f})",
            line_color=colors[i % len(colors)],
            cm_labels=["Control", "ASD"]
        )

        # Extract the first trace (ROC curve) and add it to our figure
        roc_trace = temp_fig.data[0]
        fig.add_trace(roc_trace)

    # Add reference line
    fig.add_trace(go.Scatter(
        x=[0, 1],
        y=[0, 1],
        mode='lines',
        line=dict(dash='dash', color='black'),
        name='Chance',
        showlegend=False
    ))

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        legend=dict(
            yanchor="bottom",
            y=0.01,
            xanchor="right",
            x=0.99
        ),
        width=900,
        height=700
    )

    return fig

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [22]:
if p_Method == "ASD-DiagNet" and p_mode == "whole":
    num_corr = len(all_corr[flist[0]][0])
    print("num_corr:  ", num_corr)

    start = time.time()
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001
    num_epochs = 25

    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr / 4)
    print("Initial n_lat:", n_lat)
    start = time.time()

    print('p_bernoulli:', p_bernoulli)
    print('augmentation:', augmentation, 'aug_factor:', aug_factor,
          'num_neighbs:', num_neighbs, 'lim4sim:', lim4sim)
    print('use_dropout:', use_dropout, '\n')



    all_predictions = {
    'fMRI Only': {'y_true': [], 'y_prob': []},
    'fMRI + Age': {'y_true': [], 'y_prob': []},
    'fMRI + All IQs': {'y_true': [], 'y_prob': []},
    'fMRI + All Features': {'y_true': [], 'y_prob': []}
    }


    feature_configs = [
    {"name": "fMRI Only", "age": False, "fiq": False, "viq": False, "piq": False},
    {"name": "fMRI + Age", "age": True, "fiq": False, "viq": False, "piq": False},
    {"name": "fMRI + All IQs", "age": False, "fiq": True, "viq": True, "piq": True},
    {"name": "fMRI + All Features", "age": True, "fiq": True, "viq": True, "piq": True}
    ]






    # Check if phenotypic data is available and optionally print a sample vector.
    if len(phen_dict) > 0:
        sample_key = list(phen_dict.keys())[0]
        pheno_vect = phen_dict[sample_key]

    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    crossval_res_kol = []
    y_arr = np.array([get_label(f) for f in flist])
    flist = np.array(flist)

    # Optionally adjust folds based on phenotypic data (as in percenter code)
    if len(phen_dict) > 0:
        unique_labels, counts = np.unique(y_arr, return_counts=True)
        new_n_splits = min(p_fold, counts.min())
        if new_n_splits < 2:
            print("Skipping due to insufficient samples per class when using phenotypic data.")
            # You could exit or set new_n_splits = 2 if that makes sense.
        else:
            print(f"Using {new_n_splits}-fold cross-validation with phenotypic data for whole dataset.")
    else:
        new_n_splits = p_fold

    for rp in range(10):
      kf = StratifiedKFold(n_splits=new_n_splits, random_state=1, shuffle=True)
      np.random.shuffle(flist)
      y_arr = np.array([get_label(f) for f in flist])
      crossval_res_kol = []

    for kk, (train_index, test_index) in enumerate(kf.split(flist, y_arr)):
        train_samples, test_samples = flist[train_index], flist[test_index]
        verbose = (True if (kk == 0) else False)
        regions_inds = get_regs(train_samples, int(num_corr / 4))
        num_inpp = len(regions_inds)
        n_lat = int(num_inpp / 2)

        # Create test loader (same for all models)
        test_loader = get_loader(data=all_corr, samples_list=test_samples,
                              batch_size=batch_size, mode='test',
                              augmentation=False, verbose=verbose,
                              regions=regions_inds, phenotype_data=phen_dict)

        # For each feature configuration
        for config in feature_configs:
            print(f"Fold {kk+1}, Config: {config['name']}")

            # Create model with specific feature configuration
            model = MTAutoEncoder(tied=True, num_inputs=num_inpp,
                                num_latent=n_lat, use_dropout=use_dropout,
                                use_age=config["age"],
                                use_fiq=config["fiq"],
                                use_viq=config["viq"],
                                use_piq=config["piq"])
            model.to(device)

            criterion_ae = nn.MSELoss(reduction='sum')
            criterion_clf = nn.BCEWithLogitsLoss()
            optimizer = optim.SGD([{'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                                 {'params': model.classifier.parameters(), 'lr': learning_rate_clf}],
                                momentum=0.9)

            train_loader = get_loader(data=all_corr, samples_list=train_samples,
                                   batch_size=batch_size, mode='train',
                                   augmentation=augmentation, aug_factor=aug_factor,
                                   num_neighbs=num_neighbs, eig_data=eig_data,
                                   similarity_fn=sim_function, verbose=verbose,
                                   regions=regions_inds, phenotype_data=phen_dict)

            # Train model
            for epoch in range(1, num_epochs + 1):
                if epoch <= 20:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                else:
                    train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')

            # Test model and get probabilities
            mlp_acc, mlp_sens, mlp_spef, y_true, y_prob = test(
                model, criterion_ae, test_loader, eval_classifier=True, return_probs=True)

            # Store metrics for traditional evaluation
            if config["name"] == "fMRI + All Features":  # Only store the full model results
                crossval_res_kol.append((mlp_acc, mlp_sens, mlp_spef))

            # Store predictions for ROC curve
            all_predictions[config["name"]]['y_true'].extend(y_true)
            all_predictions[config["name"]]['y_prob'].extend(y_prob)

    # Print average results for the full model (as before)
    print("Averages over folds:")
    print(np.mean(np.array(crossval_res_kol), axis=0))
    finish = time.time()
    print("Elapsed time:", finish - start)

# After all cross-validation, create ROC plot
fig = plot_roc_curves(all_predictions)
fig.write_html("asd_diagnet_feature_comparison.html")
fig.show()


num_corr:   19900
Initial n_lat: 4975
p_bernoulli: None
augmentation: True aug_factor: 2 num_neighbs: 5 lim4sim: 2
use_dropout: False 

Using 10-fold cross-validation with phenotypic data for whole dataset.
Fold 1, Config: fMRI Only
Classifier using No phenotypic features, shape: torch.Size([8, 4975])
Fold 1, Config: fMRI + Age
Classifier using 1 phenotypic features, shape: torch.Size([8, 4976])
Fold 1, Config: fMRI + All IQs
Classifier using 3 phenotypic features, shape: torch.Size([8, 4978])
Fold 1, Config: fMRI + All Features
Classifier using 4 phenotypic features, shape: torch.Size([8, 4979])
Fold 2, Config: fMRI Only
Classifier using No phenotypic features, shape: torch.Size([8, 4975])
Fold 2, Config: fMRI + Age
Classifier using 1 phenotypic features, shape: torch.Size([8, 4976])
Fold 2, Config: fMRI + All IQs
Classifier using 3 phenotypic features, shape: torch.Size([8, 4978])
Fold 2, Config: fMRI + All Features
Classifier using 4 phenotypic features, shape: torch.Size([8, 4979])

In [ ]:
p_ROI = "cc200"
p_fold = 5  # Use 5-fold for intra-site evaluation
p_center = "Stanford"
p_mode = "percenter"  # Change to "percenter" for intra-site evaluation
p_augmentation = True
p_Method = "ASD-DiagNet"

In [ ]:

if p_Method == "ASD-DiagNet" and p_mode == "percenter":
    num_corr = len(all_corr[flist[0]][0])

    if len(phen_dict) > 0:
        sample_key = list(phen_dict.keys())[0]
        pheno_vect = phen_dict[sample_key]

    flist = os.listdir(data_main_path)

    for f in range(len(flist)):
        flist[f] = get_key(flist[f])

    centers_dict = {}
    for f in flist:
        key = f.split('_')[0]

        if key not in centers_dict:
            centers_dict[key] = []
        centers_dict[key].append(f)

    flist = np.array(centers_dict[p_center])
    y_arr = np.array([get_label(f) for f in flist])

    # Determine the number of splits.
    if len(phen_dict) > 0:
        unique_labels, counts = np.unique(y_arr, return_counts=True)
        new_n_splits = min(p_fold, counts.min())
        if new_n_splits < 2:
            print(f"Skipping center {p_center} due to insufficient samples in one class.")
            # Optionally, exit or continue to the next center.
        else:
            print(f"Using {new_n_splits}-fold cross-validation for center {p_center}.")
    else:
        new_n_splits = p_fold
        print(f"Using {new_n_splits}-fold cross-validation for center {p_center}.")

    flist = np.array(centers_dict[p_center])

    start =time.time()
    #flist = np.array(sorted(os.listdir(data_main_path)))
    batch_size = 8
    learning_rate_ae, learning_rate_clf = 0.0001, 0.0001

    num_epochs = 25
    p_bernoulli = None
    augmentation = p_augmentation
    use_dropout = False

    aug_factor = 2
    num_neighbs = 5
    lim4sim = 2
    n_lat = int(num_corr / 4)

    print('p_bernoulli: ', p_bernoulli)
    print('augmentation: ', augmentation, 'aug_factor: ', aug_factor,
          'num_neighbs: ', num_neighbs, 'lim4sim: ', lim4sim)
    print('use_dropout: ', use_dropout, '\n')

    sim_function = functools.partial(cal_similarity, lim=lim4sim)
    all_rp_res = []
    y_arr = np.array([get_label(f) for f in flist])


    all_predictions = {
    'fMRI Only': {'y_true': [], 'y_prob': []},
    'fMRI + Age': {'y_true': [], 'y_prob': []},
    'fMRI + All IQs': {'y_true': [], 'y_prob': []},
    'fMRI + All Features': {'y_true': [], 'y_prob': []}
    }

    # Feature configurations to test
    feature_configs = [
    {"name": "fMRI Only", "age": False, "fiq": False, "viq": False, "piq": False},
    {"name": "fMRI + Age", "age": True, "fiq": False, "viq": False, "piq": False},
    {"name": "fMRI + All IQs", "age": False, "fiq": True, "viq": True, "piq": True},
    {"name": "fMRI + All Features", "age": True, "fiq": True, "viq": True, "piq": True}
    ]

    # Only continue if we have enough splits.
    if len(phen_dict) > 0 and new_n_splits < 2:
        pass
    else:
        for rp in range(10):
            print("========================")
            crossval_res_kol = []
            start_rp = time.time()
            kf = StratifiedKFold(n_splits=new_n_splits)

            for kk, (train_index, test_index) in enumerate(kf.split(flist, y_arr)):
                train_samples, test_samples = flist[train_index], flist[test_index]
                verbose = (True if (kk == 0) else False)

                regions_inds = get_regs(train_samples, int(num_corr/4))
                num_inpp = len(regions_inds)
                n_lat = int(num_inpp/2)

                # Create test loader (same for all models)
                test_loader = get_loader(data=all_corr, samples_list=test_samples,
                                         batch_size=batch_size, mode='test',
                                         augmentation=False, verbose=verbose,
                                         regions=regions_inds, phenotype_data=phen_dict)

                # For each feature configuration
                for config in feature_configs:
                    # Print which configuration is being processed
                    if verbose:
                        print(f"  Processing {config['name']}")

                    # Create model with specific feature configuration
                    model = MTAutoEncoder(
                        num_inputs=num_inpp,
                        num_latent=n_lat,
                        tied=True,
                        use_dropout=use_dropout,
                        use_age=config["age"],
                        use_fiq=config["fiq"],
                        use_viq=config["viq"],
                        use_piq=config["piq"]
                    )
                    model.to(device)

                    criterion_ae = nn.MSELoss(reduction='sum')
                    criterion_clf = nn.BCEWithLogitsLoss()
                    optimizer = optim.SGD([
                        {'params': model.fc_encoder.parameters(), 'lr': learning_rate_ae},
                        {'params': model.classifier.parameters(), 'lr': learning_rate_clf}
                    ], momentum=0.9)

                    # Create train loader
                    train_loader = get_loader(data=all_corr, samples_list=train_samples,
                                              batch_size=batch_size, mode='train',
                                              augmentation=augmentation, aug_factor=aug_factor,
                                              num_neighbs=num_neighbs, eig_data=eig_data,
                                              similarity_fn=sim_function, verbose=verbose,
                                              regions=regions_inds, phenotype_data=phen_dict)

                    # Train model
                    for epoch in range(1, num_epochs + 1):
                        if epoch <= 20:
                            train_losses = train(model, epoch, train_loader, p_bernoulli, mode='both')
                        else:
                            train_losses = train(model, epoch, train_loader, p_bernoulli, mode='clf')

                    # Evaluate model and get probabilities
                    mlp_acc, mlp_sens, mlp_spef, y_true, y_prob = test(
                        model, criterion_ae, test_loader, eval_classifier=True, return_probs=True)

                    # Store metrics for traditional evaluation (only for full model)
                    if config["name"] == "fMRI + All Features":
                        crossval_res_kol.append((mlp_acc, mlp_sens, mlp_spef))

                    # Store predictions for ROC curve
                    all_predictions[config["name"]]['y_true'].extend(y_true)
                    all_predictions[config["name"]]['y_prob'].extend(y_prob)

            print("Result of repeat ", rp, ":")
            print(np.mean(np.array(crossval_res_kol), axis=0))
            all_rp_res.append(np.mean(np.array(crossval_res_kol), axis=0))
            finish_rp = time.time()
            print("Running time:", finish_rp - start_rp)

        print("Average result of 10 repeats: ", np.mean(np.array(all_rp_res), axis=0))

        # After all repeats, create ROC plot
        fig = plot_roc_curves(all_predictions)
        fig.write_html(f"asd_diagnet_{p_center}_feature_comparison.html")
        fig.show()

Using 5-fold cross-validation for center Stanford.
p_bernoulli:  None
augmentation:  True aug_factor:  2 num_neighbs:  5 lim4sim:  2
use_dropout:  False 

  Processing fMRI Only
Classifier using No phenotypic features, shape: torch.Size([8, 4975])
  Processing fMRI + Age
Classifier using 1 phenotypic features, shape: torch.Size([8, 4976])
  Processing fMRI + All IQs
Classifier using 3 phenotypic features, shape: torch.Size([8, 4978])
  Processing fMRI + All Features
Classifier using 4 phenotypic features, shape: torch.Size([8, 4979])
Classifier using No phenotypic features, shape: torch.Size([8, 4975])
Classifier using 1 phenotypic features, shape: torch.Size([8, 4976])
Classifier using 3 phenotypic features, shape: torch.Size([8, 4978])
Classifier using 4 phenotypic features, shape: torch.Size([8, 4979])
Classifier using No phenotypic features, shape: torch.Size([8, 4975])
Classifier using 1 phenotypic features, shape: torch.Size([8, 4976])
Classifier using 3 phenotypic features, shap